In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우 (입력값 =/= 출력값 dim)
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False) #conv layer 1개
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1) #basic block 당 conv layer 2개
                                                                            #그게 2개이므로 총 4개
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 128, num_blocks[3], stride=2)
        #총 16개의 conv layer
        
        self.linear = nn.Linear(128, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)#첫번째 conv 연산에 의해서만 너비와 높이가 줄어들 수 있도록
                                                   #나머지는 stride = 1 로하여 같은 너비와 높이 유지
                                                #filter 개수가 증가할때마다 너비와 높이는 줄어들수 있게 설정
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [2]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
  
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
device = 'cuda:1'

net = ResNet18()
net = net.to(device)
# net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')

#learning rate를 바꾸기
def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10 #1/10 로 줄이기
    if epoch >= 150:
        lr /= 10  #1/10 로 줄이기
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [4]:
for epoch in range(0, 200):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.046875
Current benign train loss: 2.417367458343506

Current batch: 100
Current benign train accuracy: 0.40625
Current benign train loss: 1.7210363149642944

Current batch: 200
Current benign train accuracy: 0.4375
Current benign train loss: 1.5398842096328735

Current batch: 300
Current benign train accuracy: 0.4453125
Current benign train loss: 1.3980908393859863

Total benign train accuarcy: 37.574
Total benign train loss: 654.2415896654129

[ Test epoch: 0 ]

Test accuarcy: 48.38
Test average loss: 0.014429092335700989
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.4765625
Current benign train loss: 1.3413275480270386

Current batch: 100
Current benign train accuracy: 0.484375
Current benign train loss: 1.2855979204177856

Current batch: 200
Current benign train accuracy: 0.5078125
Current benign train loss: 1.280807614326477

Current batch: 300
Current benign train accuracy:


Current batch: 200
Current benign train accuracy: 0.8671875
Current benign train loss: 0.43387964367866516

Current batch: 300
Current benign train accuracy: 0.7890625
Current benign train loss: 0.5157380700111389

Total benign train accuarcy: 85.348
Total benign train loss: 164.08392375707626

[ Test epoch: 13 ]

Test accuarcy: 82.61
Test average loss: 0.005267876473069191
Model Saved!

[ Train epoch: 14 ]

Current batch: 0
Current benign train accuracy: 0.875
Current benign train loss: 0.3294738233089447

Current batch: 100
Current benign train accuracy: 0.90625
Current benign train loss: 0.25019487738609314

Current batch: 200
Current benign train accuracy: 0.8828125
Current benign train loss: 0.3614737391471863

Current batch: 300
Current benign train accuracy: 0.90625
Current benign train loss: 0.2638549208641052

Total benign train accuarcy: 86.044
Total benign train loss: 157.10601633787155

[ Test epoch: 14 ]

Test accuarcy: 81.07
Test average loss: 0.005748622864484787
Model 


Total benign train accuarcy: 89.124
Total benign train loss: 121.67169962823391

[ Test epoch: 26 ]

Test accuarcy: 85.49
Test average loss: 0.0043461643561720845
Model Saved!

[ Train epoch: 27 ]

Current batch: 0
Current benign train accuracy: 0.9453125
Current benign train loss: 0.16676108539104462

Current batch: 100
Current benign train accuracy: 0.859375
Current benign train loss: 0.37711623311042786

Current batch: 200
Current benign train accuracy: 0.8828125
Current benign train loss: 0.3318544328212738

Current batch: 300
Current benign train accuracy: 0.8359375
Current benign train loss: 0.34869492053985596

Total benign train accuarcy: 88.918
Total benign train loss: 122.23738047480583

[ Test epoch: 27 ]

Test accuarcy: 85.47
Test average loss: 0.0043701958566904064
Model Saved!

[ Train epoch: 28 ]

Current batch: 0
Current benign train accuracy: 0.9140625
Current benign train loss: 0.1832515150308609

Current batch: 100
Current benign train accuracy: 0.859375
Current ben


Current batch: 0
Current benign train accuracy: 0.9140625
Current benign train loss: 0.22489389777183533

Current batch: 100
Current benign train accuracy: 0.921875
Current benign train loss: 0.2654535472393036

Current batch: 200
Current benign train accuracy: 0.8828125
Current benign train loss: 0.3837480843067169

Current batch: 300
Current benign train accuracy: 0.90625
Current benign train loss: 0.28218477964401245

Total benign train accuarcy: 90.62
Total benign train loss: 103.30661328136921

[ Test epoch: 40 ]

Test accuarcy: 84.22
Test average loss: 0.004967880339920521
Model Saved!

[ Train epoch: 41 ]

Current batch: 0
Current benign train accuracy: 0.859375
Current benign train loss: 0.36224979162216187

Current batch: 100
Current benign train accuracy: 0.90625
Current benign train loss: 0.24737128615379333

Current batch: 200
Current benign train accuracy: 0.8671875
Current benign train loss: 0.275439977645874

Current batch: 300
Current benign train accuracy: 0.890625
Cu


Current batch: 200
Current benign train accuracy: 0.8984375
Current benign train loss: 0.25339287519454956

Current batch: 300
Current benign train accuracy: 0.9453125
Current benign train loss: 0.15669503808021545

Total benign train accuarcy: 91.512
Total benign train loss: 93.37656472623348

[ Test epoch: 53 ]

Test accuarcy: 85.01
Test average loss: 0.004884416474401951
Model Saved!

[ Train epoch: 54 ]

Current batch: 0
Current benign train accuracy: 0.921875
Current benign train loss: 0.24251016974449158

Current batch: 100
Current benign train accuracy: 0.9296875
Current benign train loss: 0.20516109466552734

Current batch: 200
Current benign train accuracy: 0.8984375
Current benign train loss: 0.20500442385673523

Current batch: 300
Current benign train accuracy: 0.9296875
Current benign train loss: 0.1680559515953064

Total benign train accuarcy: 91.464
Total benign train loss: 93.72830464690924

[ Test epoch: 54 ]

Test accuarcy: 87.37
Test average loss: 0.00388844810575246

...


Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.028736736625432968

Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02483103796839714

Current batch: 300
Current benign train accuracy: 0.9765625
Current benign train loss: 0.0348973348736763

Total benign train accuarcy: 99.156
Total benign train loss: 9.848971795290709

[ Test epoch: 133 ]

Test accuarcy: 91.82
Test average loss: 0.0036886039175093176
Model Saved!

[ Train epoch: 134 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.014957502484321594

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.03672534599900246

Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.026021219789981842

Current batch: 300
Current benign train accuracy: 0.984375
Current benign train loss: 0.05792424827814102

Total benign train accuarcy: 99.174
Total ben


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.006489802151918411

Total benign train accuarcy: 99.344
Total benign train loss: 8.17771552875638

[ Test epoch: 146 ]

Test accuarcy: 91.37
Test average loss: 0.003938963939994574
Model Saved!

[ Train epoch: 147 ]

Current batch: 0
Current benign train accuracy: 0.984375
Current benign train loss: 0.04649203270673752

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.027164366096258163

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.005631573498249054

Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.019279591739177704

Total benign train accuarcy: 99.288
Total benign train loss: 8.20858146995306

[ Test epoch: 147 ]

Test accuarcy: 91.62
Test average loss: 0.0038746543943881987
Model Saved!

[ Train epoch: 148 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign tr


Test accuarcy: 92.01
Test average loss: 0.0037830126903951166
Model Saved!

[ Train epoch: 160 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.010583363473415375

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.007749617099761963

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.011021532118320465

Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02269042655825615

Total benign train accuarcy: 99.642
Total benign train loss: 5.1672773994505405

[ Test epoch: 160 ]

Test accuarcy: 92.02
Test average loss: 0.0038147728905081747
Model Saved!

[ Train epoch: 161 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.013153359293937683

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.008462212979793549

Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train l


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.01023980975151062

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.00629083439707756

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0037034600973129272

Total benign train accuarcy: 99.672
Total benign train loss: 4.747534669935703

[ Test epoch: 173 ]

Test accuarcy: 91.99
Test average loss: 0.0038467231795191767
Model Saved!

[ Train epoch: 174 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.004850979894399643

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0054443515837192535

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.006447680294513702

Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.012867249548435211

Total benign train accuarcy: 99.658
Total benign train loss: 4.884204094298


Total benign train accuarcy: 99.7
Total benign train loss: 4.354186460375786

[ Test epoch: 186 ]

Test accuarcy: 91.95
Test average loss: 0.00383486949428916
Model Saved!

[ Train epoch: 187 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.006153341382741928

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.008396290242671967

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.011229299008846283

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.007675398141145706

Total benign train accuarcy: 99.73
Total benign train loss: 4.156634753569961

[ Test epoch: 187 ]

Test accuarcy: 91.91
Test average loss: 0.003865399669110775
Model Saved!

[ Train epoch: 188 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.003233574330806732

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0082825943827